# Using FAISS and Lamini for RAG Pipeline

## Step 1: Import Libraries
Import the necessary libraries, including FAISS, Lamini, and jsonlines to handle embeddings and data processing.

In [1]:
import faiss
import lamini
import jsonlines

## Step 2: Set Parameters
Define parameters like the number of nearest neighbors to be returned.

In [2]:
# Number of nearest chunks to return
k = 2

## Step 3: Initialize Variables
Set up placeholders for the index and corresponding plain text splits. Instantiate Lamini's embedding client.

In [12]:
# Set up for the index, which holds the embeddings, and the splits, which holds the corresponding plain text
index = None
splits = []

# Instantiate Lamini's embedding client
embedding_client = lamini.Embedding()

## Step 4: Create Embeddings for Each Transcript
Load each 'transcript' item from a JSONL file, create an embedding for it, and add it to the index.

In [5]:
# Load each 'transcript' item and create embeddings
with jsonlines.open("data.jsonl", "r") as file:
    for item in file:
        transcript_embedding = embedding_client.generate(item['transcript'])
        if not index:
            index = faiss.IndexFlatL2(transcript_embedding.size) # Set the size of the index based on model embedding size
        index.add(transcript_embedding)
        splits.append(item['transcript'])

## Step 5: Create Embedding for the Question
Create an embedding for the user's question to find the most relevant text from the index.

In [6]:
# Define the question and create its embedding
question = "What is TSMC's 2019 revenue in USD?"
question_embedding = embedding_client.generate(question)

## Step 6: Find Nearest Neighbors
Use FAISS to find the top ( k ) nearest neighbors for the question embedding.

In [7]:
# Find the k nearest neighbors and retrieve relevant data
distances, indices = index.search(question_embedding, k)
relevant_data = [splits[i] for i in indices[0] if i >= 0]

## Step 7: Instantiate LLM Client
Initialize Lamini's LLM client for generating the final response based on the relevant data.

In [8]:
# Instantiate Lamini's LLM client
llm = lamini.Lamini(model_name="meta-llama/Meta-Llama-3.1-8B-Instruct")

## Step 8: Prepare the Prompt
Construct a prompt for Lamini's LLM using the retrieved relevant data and the question.

In [9]:
# Form the prompt using the retrieved data and the question
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{relevant_data}
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

## Step 9: Generate the Response
Use the constructed prompt to generate the answer to the user's question with Lamini's LLM.

In [14]:
# Generate the answer using Lamini's LLM
response = llm.generate(prompt)
print(response)



The text does not mention TSMC (Taiwan Semiconductor Manufacturing Company) at all. It appears to be a transcript of a conference call from a company that is not specified. However, based on the content, it seems to be a company in the semiconductor industry, possibly a supplier to TSMC.

The text mentions a revenue range of $550 million to $565 million for 2019, but it does not specify the company's name.
